In [1]:
import numpy
import torch
import mobster as mb

import numpy as np


In [2]:
data = mb.load_example_data("./")

In [5]:
import pyro

pyro.enable_validation(True)
fitted = mb.fit_mobster(data,1,1, subclonal_prior="Moyal",purity = 1,  truncated_pareto=True, max_it = 300, lr = 0.01,  e = 0.0000005, lrd_gamma = 0.1,
        number_of_trials_k= 1000, prior_lims_k=[100,1000000], prior_lims_clonal=[100,1000000.], number_of_trials_clonal_mean= 1000)


Running MOBSTER on 1 karyotypes with 1 subclones.
Fitting a model with tail


ELBO: 1895.364854318  : 100%|██████████| 300/300 [00:03<00:00, 95.00it/s] 

{'multitail_weights': tensor([[0.4325, 0.5675]], grad_fn=<DivBackward0>), 'tail_mean': tensor([0.9182], grad_fn=<AddBackward0>), 'alpha_noise': tensor([49.8206], grad_fn=<AddBackward0>), 'avg_number_of_trials_beta': tensor([1184.4547], grad_fn=<AddBackward0>), 'prc_number_of_trials_beta': tensor([13.7204], grad_fn=<AddBackward0>), 'ccf_priors': tensor([0.3794], grad_fn=<ClampBackward1>), 'param_tail_weights': tensor([[0.2920, 0.7080]], grad_fn=<DivBackward0>), 'param_weights_1': tensor([[0.7849, 0.2151]], grad_fn=<DivBackward0>), 'a_1': tensor([[0.4929]], grad_fn=<ClampBackward1>), 'n_trials_subclonal_0': tensor([299.9209], requires_grad=True)}
Computing cluster assignements.


Computing information criteria.
Done!



In [ ]:


mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [6]:
fitted["model_parameters"]["1:1"]



{'cluster_probs': array([[9.8495191e-01, 9.8228878e-01, 9.7440338e-01, ..., 3.7723485e-01,
         4.0441286e-02, 8.9967065e-02],
        [1.2972568e-19, 4.2881639e-18, 3.0994127e-18, ..., 6.2274659e-01,
         9.5955878e-01, 9.1003293e-01],
        [1.5048128e-02, 1.7711407e-02, 2.5596738e-02, ..., 1.8666633e-05,
         2.6537437e-11, 1.5042492e-09]], dtype=float32),
 'cluster_assignments': array(['Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail', 'Tail',
        'S1', 'Tail', 'Ta

In [18]:
moy.rsample(torch.ones(3).size())

tensor([ 6.5504e+00, -2.3366e-03,  1.9606e+00])